# Phase 0.1: Research Korean Medical Datasets

Research and document available datasets for Korean medical LLM training.

## Contents
1. Korean Medical Datasets
2. Korean General Corpus
3. Korean Instruction Datasets
4. Bilingual Resources
5. Dataset Summary and Selection

In [1]:
# Setup
import sys
sys.path.append("..")

from datasets import load_dataset
from huggingface_hub import HfApi, list_datasets
import pandas as pd

api = HfApi()

/home/ormastes/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


---
## 1. Korean Medical Datasets

### 1.1 KorMedMCQA (Primary Benchmark)

**Source**: [sean0042/KorMedMCQA](https://huggingface.co/datasets/sean0042/KorMedMCQA)

**Description**: First Korean Medical Multiple-Choice Question Answering benchmark from professional healthcare licensing examinations (2012-2024).

**Details**:
- 7,469 questions from doctor, nurse, pharmacist, dentist exams
- Wide range of medical disciplines
- Best model: o1-preview (92.72%), Qwen2.5-72B (78.86%)
- Chain of Thought improves performance by up to 4.5%

**Paper**: [arXiv:2403.01469](https://arxiv.org/abs/2403.01469)

In [2]:
# Explore KorMedMCQA - load just doctor config to save memory
print("KorMedMCQA Dataset Structure:")
print("Available configs: dentist, doctor, nurse, pharm")

try:
    kormedmcqa = load_dataset("sean0042/KorMedMCQA", "doctor")
    print(f"\nDoctor config loaded:")
    print(kormedmcqa)
    
    print("\nSample entry:")
    sample = kormedmcqa['train'][0] if 'train' in kormedmcqa else kormedmcqa[list(kormedmcqa.keys())[0]][0]
    for key, value in sample.items():
        print(f"  {key}: {str(value)[:200]}..." if len(str(value)) > 200 else f"  {key}: {value}")
except Exception as e:
    print(f"Could not load KorMedMCQA: {e}")

KorMedMCQA Dataset Structure:
Available configs: dentist, doctor, nurse, pharm



Doctor config loaded:
DatasetDict({
    train: Dataset({
        features: ['subject', 'year', 'period', 'q_number', 'question', 'A', 'B', 'C', 'D', 'E', 'answer', 'cot'],
        num_rows: 1890
    })
    dev: Dataset({
        features: ['subject', 'year', 'period', 'q_number', 'question', 'A', 'B', 'C', 'D', 'E', 'answer', 'cot'],
        num_rows: 164
    })
    test: Dataset({
        features: ['subject', 'year', 'period', 'q_number', 'question', 'A', 'B', 'C', 'D', 'E', 'answer', 'cot'],
        num_rows: 435
    })
    fewshot: Dataset({
        features: ['subject', 'year', 'period', 'q_number', 'question', 'A', 'B', 'C', 'D', 'E', 'answer', 'cot'],
        num_rows: 5
    })
})

Sample entry:
  subject: doctor
  year: 2012
  period: 1
  q_number: 1
  question: 항문압 측정 검사에서 항문 압력이 증가하는 경우는?
  A: 직장질루(rectovaginal fistula)
  B: 항문열창(anal fissure)
  C: 대변실금(fecal incontinence)
  D: 대변메막힘(fecal impaction)
  E: 직장탈출증(rectal prolapse)
  answer: 2
  cot: 


In [3]:
# Dataset statistics for loaded config
print("Dataset Statistics (doctor config):")
if 'kormedmcqa' in dir():
    for split in kormedmcqa.keys():
        print(f"  {split}: {len(kormedmcqa[split])} examples")
    print(f"\nColumns: {kormedmcqa['train'].column_names if 'train' in kormedmcqa else 'N/A'}")
    print("\nNote: Total across all configs (dentist, doctor, nurse, pharm) is ~7,469 examples")
    
    # Clean up memory
    del kormedmcqa
    import gc
    gc.collect()

Dataset Statistics (doctor config):
  train: 1890 examples
  dev: 164 examples
  test: 435 examples
  fewshot: 5 examples

Columns: ['subject', 'year', 'period', 'q_number', 'question', 'A', 'B', 'C', 'D', 'E', 'answer', 'cot']

Note: Total across all configs (dentist, doctor, nurse, pharm) is ~7,469 examples


### 1.2 KorMedLawQA

**Source**: [snuh/KorMedLawQA](https://huggingface.co/datasets/snuh/KorMedLawQA)

**Description**: Korean Medical Law QA dataset from Seoul National University Hospital (SNUH).

In [4]:
# Explore KorMedLawQA - skip if not available
print("KorMedLawQA Dataset:")
print("  Source: snuh/KorMedLawQA")
print("  Note: May require authentication or special access")
print("  Skipping load to save memory - check manually if needed")

KorMedLawQA Dataset:
  Source: snuh/KorMedLawQA
  Note: May require authentication or special access
  Skipping load to save memory - check manually if needed


### 1.3 Medical Reasoning KorMedMCQA

**Source**: [ChuGyouk/medical-reasoning-train-kormedmcqa](https://huggingface.co/datasets/ChuGyouk/medical-reasoning-train-kormedmcqa)

**Description**: KorMedMCQA with reasoning chains for Chain-of-Thought training.

In [5]:
# Explore Medical Reasoning dataset - brief check
print("Medical Reasoning KorMedMCQA:")
print("  Source: ChuGyouk/medical-reasoning-train-kormedmcqa")
print("  Description: KorMedMCQA with Chain-of-Thought reasoning")
print("  Skipping full load to save memory")

Medical Reasoning KorMedMCQA:
  Source: ChuGyouk/medical-reasoning-train-kormedmcqa
  Description: KorMedMCQA with Chain-of-Thought reasoning
  Skipping full load to save memory


### 1.4 KBMC (Korean Bio-Medical Corpus for NER)

**Source**: [arXiv:2403.16158](https://arxiv.org/abs/2403.16158)

**Description**: First open-source Korean medical NER dataset.

**Details**:
- Entity types: disease name, body part, treatment
- BIO format annotations
- 20% improvement over general Korean NER datasets
- Published at LREC-COLING 2024

In [6]:
# Search for KBMC on HuggingFace
kbmc_datasets = list(api.list_datasets(search="KBMC Korean medical", limit=10))
print("KBMC related datasets on HuggingFace:")
for ds in kbmc_datasets:
    print(f"  - {ds.id}")

if not kbmc_datasets:
    print("  No KBMC datasets found on HuggingFace.")
    print("  May need to request from paper authors or construct manually.")

KBMC related datasets on HuggingFace:
  No KBMC datasets found on HuggingFace.
  May need to request from paper authors or construct manually.


---
## 2. Korean General Corpus (for Tokenizer Training)

### 2.1 OSCAR Korean

**Source**: [oscar-corpus/OSCAR-2301](https://huggingface.co/datasets/oscar-corpus/OSCAR-2301)

**Description**: Open Super-large Crawled Aggregated coRpus - multilingual web corpus.

**Details**:
- Korean subset available (`ko` language code)
- Large scale (10GB+)
- Research-only license

In [7]:
# Explore OSCAR Korean (streaming due to size)
print("OSCAR-2301 Korean subset:")
print("  Source: oscar-corpus/OSCAR-2301")
print("  Note: This is a GATED dataset requiring HuggingFace authentication")
print("  To access: 1) Login at huggingface.co")
print("            2) Accept terms at https://huggingface.co/datasets/oscar-corpus/OSCAR-2301")
print("            3) Run: huggingface-cli login")
print("\nAlternative: Use cc100 or other open Korean corpora")

OSCAR-2301 Korean subset:
  Source: oscar-corpus/OSCAR-2301
  Note: This is a GATED dataset requiring HuggingFace authentication
  To access: 1) Login at huggingface.co
            2) Accept terms at https://huggingface.co/datasets/oscar-corpus/OSCAR-2301
            3) Run: huggingface-cli login

Alternative: Use cc100 or other open Korean corpora


### 2.2 mC4 Korean

**Source**: [mc4](https://huggingface.co/datasets/mc4) (deprecated, use [allenai/c4](https://huggingface.co/datasets/allenai/c4))

**Description**: Multilingual colossal cleaned Common Crawl corpus.

**Details**:
- 108 languages supported
- Korean subset available
- Very large scale

In [8]:
# Explore mC4 Korean (streaming)
print("mC4 Korean subset:")
print("  Source: mc4 (ko config)")
print("  Note: May also require authentication")
print("  Alternative: allenai/c4")
print("\nSkipping load - check access requirements manually")

mC4 Korean subset:
  Source: mc4 (ko config)
  Note: May also require authentication
  Alternative: allenai/c4

Skipping load - check access requirements manually


### 2.3 Korean Pretraining Dataset Collection

**Source**: [heegyu/korean-pretraining-dataset](https://huggingface.co/collections/heegyu/korean-pretraining-dataset-65c59136735dd9c8163ec50c)

**Description**: Curated collection of Korean pretraining datasets including mC4 + OSCAR.

In [9]:
# Search for Korean pretraining datasets
ko_pretrain_datasets = list(api.list_datasets(search="Korean pretraining", limit=20))
print("Korean pretraining datasets on HuggingFace:")
for ds in ko_pretrain_datasets:
    print(f"  - {ds.id}: {ds.downloads} downloads")

Korean pretraining datasets on HuggingFace:
  - pretraining/tiny-korean-100k: 14 downloads
  - oz1115/korean-pretraining-corpus: 16 downloads
  - oz1115/korean-pretraining-corpus-ko: 12 downloads


### 2.4 Korean Wikipedia

**Source**: [wikimedia/wikipedia](https://huggingface.co/datasets/wikimedia/wikipedia)

**Description**: Wikipedia dumps for multiple languages.

In [10]:
# Explore Korean Wikipedia
print("Korean Wikipedia:")
print("  Source: wikimedia/wikipedia (20231101.ko)")

try:
    wiki_ko = load_dataset("wikimedia/wikipedia", "20231101.ko", split="train", streaming=True)
    
    print("\nSample entries:")
    for i, example in enumerate(wiki_ko):
        print(f"\nSample {i+1}:")
        print(f"  Title: {example.get('title', 'N/A')}")
        text_preview = example['text'][:300] if 'text' in example else str(example)[:300]
        print(f"  Text (first 300 chars): {text_preview}...")
        if i >= 1:
            break
except Exception as e:
    print(f"Could not load Korean Wikipedia: {e}")
    print("May need to check dataset path or authentication")

Korean Wikipedia:
  Source: wikimedia/wikipedia (20231101.ko)



Sample entries:



Sample 1:
  Title: 지미 카터
  Text (first 300 chars): 제임스 얼 카터 주니어(, 1924년 10월 1일~)는 민주당 출신 미국의 제39대 대통령(1977년~1981년)이다.

생애

어린 시절 
지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.

조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.

정계 입문 
1962년 조지아주 상원 의원 선거에서 낙선하였으나, 그 선거가 부...

Sample 2:
  Title: 수학
  Text (first 300 chars): 수학(數學, , 줄여서 math)은 수, 양, 구조, 공간, 변화 등의 개념을 다루는 학문이다. 널리 받아들여지는 명확한 정의는 없으나 현대 수학은 일반적으로 엄밀한 논리에 근거하여 추상적 대상을 탐구하며, 이는 규칙의 발견과 문제의 제시 및 해결의 과정으로 이루어진다. 수학은 그 발전 과정에 있어서 철학, 과학과 깊은 연관을 맺고 있으며, 엄밀한 논리와 특유의 추상성, 보편성에 의해 다른 학문들과 구별된다. 특히 수학은 과학의 여느 분야들과는 달리 자연계에서 관측되지 않는 개념들에 대해서까지 이론을 추상화시키는 특징을 보이는데, ...


---
## 3. Korean Instruction Datasets

### 3.1 KoAlpaca

**Source**: [beomi/KoAlpaca-v1.1a](https://huggingface.co/datasets/beomi/KoAlpaca-v1.1a)

**Description**: Korean instruction-following dataset based on Stanford Alpaca.

**Details**:
- 21,155 examples
- Cost: < $500 to create
- Features: instruction, input, output

In [11]:
# Explore KoAlpaca - streaming to save memory
print("KoAlpaca Dataset:")
print("  Source: beomi/KoAlpaca-v1.1a")
print("  Size: 21,155 examples")
print("  Features: instruction, input, output")

try:
    koalpaca = load_dataset("beomi/KoAlpaca-v1.1a", split="train", streaming=True)
    print("\nSample entries:")
    for i, example in enumerate(koalpaca):
        print(f"\n--- Sample {i+1} ---")
        print(f"Instruction: {example['instruction'][:200]}...")
        print(f"Output: {example['output'][:200]}...")
        if i >= 1:
            break
except Exception as e:
    print(f"Could not load: {e}")

KoAlpaca Dataset:
  Source: beomi/KoAlpaca-v1.1a
  Size: 21,155 examples
  Features: instruction, input, output



Sample entries:



--- Sample 1 ---
Instruction: 양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?...
Output: 양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. 

식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.

 ...

--- Sample 2 ---
Instruction: 스웨터의 유래는 어디에서 시작되었나요?...
Output: 스웨터의 유래는 14세기경 북유럽항구지역에서 어망을 짜던 기술을 의복에 활용하면서 시작되었습니다. 노동자들의 방한복에서 시작된 스웨터는 여가생활과 스포츠의 붐에 힘입어 대중화되었습니다. 이후, 겨울철 이너웨어의 대명사가 되었습니다. 스웨터는 짜서(Knit) 만든 옷을 말하며, 어부들의 방한복으로 짜여졌던 스웨터 중에서도 스코틀랜드 해안지방의 여인들은 바다로...


### 3.2 KoAlpaca-RealQA

**Source**: [beomi/KoAlpaca-RealQA](https://huggingface.co/datasets/beomi/KoAlpaca-RealQA)

**Description**: Real Korean user interactions from ChatKoAlpaca service (2023-2024), with GPT-4o generated answers.

In [12]:
# KoAlpaca-RealQA info
print("KoAlpaca-RealQA Dataset:")
print("  Source: beomi/KoAlpaca-RealQA")
print("  Description: Real user interactions with GPT-4o answers")
print("  Skipping load to save memory")

KoAlpaca-RealQA Dataset:
  Source: beomi/KoAlpaca-RealQA
  Description: Real user interactions with GPT-4o answers
  Skipping load to save memory


### 3.3 Korean Translated Alpaca

**Source**: [Bingsu/ko_alpaca_data](https://huggingface.co/datasets/Bingsu/ko_alpaca_data)

**Description**: Korean translation of Alpaca data via DeepL API, with GPT-3.5-turbo generated outputs.

In [13]:
# ko_alpaca_data info
print("Bingsu/ko_alpaca_data:")
print("  Size: 49,620 examples")
print("  Features: instruction, input, output")
print("  Description: Korean translation via DeepL API")
print("  Skipping load to save memory")

Bingsu/ko_alpaca_data:
  Size: 49,620 examples
  Features: instruction, input, output
  Description: Korean translation via DeepL API
  Skipping load to save memory


---
## 4. Bilingual Resources

### 4.1 UMLS Korean Mappings

**Description**: Unified Medical Language System contains Korean translations for medical terminology.

**Access**: Requires UMLS license from NLM

In [14]:
# Note: UMLS requires license
print("UMLS Korean Medical Terminology:")
print("  - Requires UMLS license from NLM (https://www.nlm.nih.gov/research/umls/)")
print("  - Contains Korean translations for medical terms")
print("  - Useful for bilingual dictionary construction")
print("\nAlternative: Create custom bilingual medical dictionary")

UMLS Korean Medical Terminology:
  - Requires UMLS license from NLM (https://www.nlm.nih.gov/research/umls/)
  - Contains Korean translations for medical terms
  - Useful for bilingual dictionary construction

Alternative: Create custom bilingual medical dictionary


### 4.2 CCAligned / OPUS Parallel Corpora

**Description**: Parallel corpora for English-Korean translation.

In [15]:
# Search for English-Korean parallel corpora
parallel_datasets = list(api.list_datasets(search="English Korean parallel translation", limit=15))
print("English-Korean parallel corpora on HuggingFace:")
for ds in parallel_datasets:
    print(f"  - {ds.id}")

English-Korean parallel corpora on HuggingFace:


---
## 5. Dataset Summary and Selection

### Selected Datasets for Korean MedGemma Training

In [16]:
# Create summary dataframe
dataset_summary = pd.DataFrame([
    {
        "Name": "KorMedMCQA",
        "Type": "Medical QA",
        "Size": "7,469 QA pairs",
        "Source": "sean0042/KorMedMCQA",
        "Use": "Evaluation + Instruction tuning",
        "Priority": "High",
    },
    {
        "Name": "Medical Reasoning KorMedMCQA",
        "Type": "Medical QA + CoT",
        "Size": "~7K",
        "Source": "ChuGyouk/medical-reasoning-train-kormedmcqa",
        "Use": "Chain-of-Thought training",
        "Priority": "High",
    },
    {
        "Name": "KorMedLawQA",
        "Type": "Medical Law QA",
        "Size": "TBD",
        "Source": "snuh/KorMedLawQA",
        "Use": "Domain-specific fine-tuning",
        "Priority": "Medium",
    },
    {
        "Name": "OSCAR Korean",
        "Type": "General Corpus",
        "Size": "10GB+",
        "Source": "oscar-corpus/OSCAR-2301 (ko)",
        "Use": "Tokenizer training",
        "Priority": "High",
    },
    {
        "Name": "Korean Wikipedia",
        "Type": "Encyclopedia",
        "Size": "~100M tokens",
        "Source": "wikimedia/wikipedia (ko)",
        "Use": "Language modeling + Medical filtering",
        "Priority": "High",
    },
    {
        "Name": "KoAlpaca-v1.1a",
        "Type": "Instruction",
        "Size": "21,155",
        "Source": "beomi/KoAlpaca-v1.1a",
        "Use": "General instruction tuning",
        "Priority": "Medium",
    },
    {
        "Name": "KoAlpaca-RealQA",
        "Type": "Real User QA",
        "Size": "TBD",
        "Source": "beomi/KoAlpaca-RealQA",
        "Use": "Real-world instruction tuning",
        "Priority": "Medium",
    },
    {
        "Name": "ko_alpaca_data",
        "Type": "Instruction (translated)",
        "Size": "49,620",
        "Source": "Bingsu/ko_alpaca_data",
        "Use": "Instruction tuning",
        "Priority": "Medium",
    },
])

print("Dataset Summary for Korean MedGemma:")
print("=" * 100)
print(dataset_summary.to_string(index=False))

Dataset Summary for Korean MedGemma:
                        Name                     Type           Size                                      Source                                   Use Priority
                  KorMedMCQA               Medical QA 7,469 QA pairs                         sean0042/KorMedMCQA       Evaluation + Instruction tuning     High
Medical Reasoning KorMedMCQA         Medical QA + CoT            ~7K ChuGyouk/medical-reasoning-train-kormedmcqa             Chain-of-Thought training     High
                 KorMedLawQA           Medical Law QA            TBD                            snuh/KorMedLawQA           Domain-specific fine-tuning   Medium
                OSCAR Korean           General Corpus          10GB+                oscar-corpus/OSCAR-2301 (ko)                    Tokenizer training     High
            Korean Wikipedia             Encyclopedia   ~100M tokens                    wikimedia/wikipedia (ko) Language modeling + Medical filtering     High
   

In [17]:
# Save dataset summary
import json
import os

os.makedirs("../data", exist_ok=True)

dataset_config = {
    "medical_datasets": [
        {
            "name": "KorMedMCQA",
            "hf_path": "sean0042/KorMedMCQA",
            "use": ["evaluation", "instruction_tuning"],
            "priority": "high",
        },
        {
            "name": "Medical Reasoning KorMedMCQA",
            "hf_path": "ChuGyouk/medical-reasoning-train-kormedmcqa",
            "use": ["chain_of_thought"],
            "priority": "high",
        },
        {
            "name": "KorMedLawQA",
            "hf_path": "snuh/KorMedLawQA",
            "use": ["domain_specific"],
            "priority": "medium",
        },
    ],
    "general_corpus": [
        {
            "name": "OSCAR Korean",
            "hf_path": "oscar-corpus/OSCAR-2301",
            "config": "ko",
            "use": ["tokenizer_training", "language_modeling"],
            "priority": "high",
        },
        {
            "name": "Korean Wikipedia",
            "hf_path": "wikimedia/wikipedia",
            "config": "20231101.ko",
            "use": ["language_modeling", "medical_filtering"],
            "priority": "high",
        },
    ],
    "instruction_datasets": [
        {
            "name": "KoAlpaca-v1.1a",
            "hf_path": "beomi/KoAlpaca-v1.1a",
            "use": ["instruction_tuning"],
            "priority": "medium",
        },
        {
            "name": "KoAlpaca-RealQA",
            "hf_path": "beomi/KoAlpaca-RealQA",
            "use": ["instruction_tuning"],
            "priority": "medium",
        },
        {
            "name": "ko_alpaca_data",
            "hf_path": "Bingsu/ko_alpaca_data",
            "use": ["instruction_tuning"],
            "priority": "medium",
        },
    ],
}

with open("../data/dataset_config.json", "w", encoding="utf-8") as f:
    json.dump(dataset_config, f, ensure_ascii=False, indent=2)

print("Dataset config saved to ../data/dataset_config.json")

Dataset config saved to ../data/dataset_config.json


### Training Data Plan

| Phase | Datasets | Purpose | Target Size |
|-------|----------|---------|-------------|
| Tokenizer Training | OSCAR Korean | Learn Korean subwords | 10GB |
| Stage 1-5 (Embeddings) | OSCAR + Wikipedia (medical) | Korean language modeling | 500M-1B tokens |
| Stage 6-7 (LoRA) | Mixed Korean + 10% English | Full adaptation | 1-2B tokens |
| Instruction Tuning | KorMedMCQA + KoAlpaca | Medical QA + General | 100K examples |

In [18]:
print("\n" + "=" * 60)
print("Dataset Research Complete!")
print("=" * 60)
print("\nNext steps:")
print("  1. Run 02_collect_korean_medical.ipynb to download medical datasets")
print("  2. Run 03_collect_bilingual_dict.ipynb to create bilingual dictionary")
print("  3. Run 04_preprocess_data.ipynb to prepare training data")


Dataset Research Complete!

Next steps:
  1. Run 02_collect_korean_medical.ipynb to download medical datasets
  2. Run 03_collect_bilingual_dict.ipynb to create bilingual dictionary
  3. Run 04_preprocess_data.ipynb to prepare training data
